In [ ]:
from lxml import etree
import pandas as pd
from datetime import date
import requests

In [ ]:
doc = etree.parse('BOE-A-2020-14372.xml')

In [ ]:
#etree.tostring(doc,pretty_print=True ,xml_declaration=True, encoding="utf-8")

In [107]:
raiz=doc.getroot()
raiz.tag

'documento'

In [108]:
len(raiz)

4

In [109]:
libro=raiz[0]
libro.tag

'metadatos'

In [110]:
libro=raiz[0]
libro[0].text

'BOE-A-2020-14372'

In [111]:
libro=raiz[1]
libro[0].tag

'{http://www.w3.org/1999/02/22-rdf-syntax-ns#}RDF'

In [112]:
libro=raiz[1]
libro[0].text

'\n            '

In [113]:
Identificador=raiz.find("metadatos/identificador")
Titulo=raiz.find("metadatos/titulo")
URL_pdf=raiz.find("metadatos/url_pdf")

In [114]:
print(Identificador.text)
print(Titulo.text)
print("https://www.boe.es" + URL_pdf.text)


BOE-A-2020-14372
Decreto-ley 35/2020, de 20 de octubre, de modificación del Decreto-ley 19/2020, de 19 de mayo, y del Decreto-ley 29/2020, de 28 de julio, en materia de adopción de medidas sociales y sanitarias para paliar los efectos de la pandemia generada por la COVID-19.
https://www.boe.es/boe/dias/2020/11/18/pdfs/BOE-A-2020-14372.pdf


In [115]:
materia=raiz.find("analisis/materias/materia")
materia.text

'Asistencia sanitaria'

In [116]:
materias=raiz.findall("analisis/materias/materia")
for materia in materias:
    print(materia.text)

Asistencia sanitaria
Asistencia social
Ayudas
Cataluña
Centros residenciales para personas con discapacidad
Epidemias
Establecimientos residenciales para la Tercera Edad
Pagos
Servicios Sociales de las Comunidades Autónomas


In [117]:
type(materias)

list

In [118]:
alertas=raiz.findall("analisis/alertas/alerta")
for alerta in alertas:
    print(alerta.text)

Asuntos sociales
Sanidad


# Recoge resumen diario del BOE de hoy

In [119]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)



Fecha de Hoy = 20201127
dia = 27
mes = 11
año = 2020


In [120]:
URL_XML_resumen =  "https://www.boe.es/diario_boe/xml.php?id=BOE-S-" + str(hoy)

In [121]:
URL_XML_resumen

'https://www.boe.es/diario_boe/xml.php?id=BOE-S-20201127'

In [122]:
#pip install requests for installation



url = URL_XML_resumen
r = requests.get(url)
#r.content
#r.text

In [123]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)
        
        
save_html(r.content, './BOEs/resumen.xml')

In [124]:
#r.encoding

In [125]:
resumen = etree.parse('./BOEs/resumen.xml')

In [126]:
raiz=resumen.getroot()
raiz.tag

'sumario'

In [127]:
seccion = raiz.findall("sumario/diario/seccion")
seccion

[]

In [128]:
secciones = raiz.findall("sumario/diario/seccion")
for seccion in secciones:
    print(seccion.text)

In [129]:
tabla_resumen = pd.DataFrame()

In [130]:
for seccion in raiz.xpath('//seccion[contains(@nombre, "I. Disposiciones generales")]'):
    nombre_seccion = seccion.xpath('@nombre')

    for departamento in seccion:
        nombre_departamento = departamento.xpath('@nombre') 

        for epigrafe in departamento:
            nombre_epigrafe = epigrafe.xpath('@nombre')

            for item in epigrafe:
                item_id = item.xpath('@id')
                item_name = item.xpath('.//titulo/text()') 

                item_urlXml = "https://www.boe.es" + str(item.xpath('.//urlXml/text()'))[2:-2]
                #print(item_urlXml)

                tabla_resumen = tabla_resumen.append({'Seccion': nombre_seccion, 
                                                      'Departamento': nombre_departamento, 
                                                      'Epigrafe' : nombre_epigrafe,
                                                      'Item_id': item_id, 
                                                      'Item_Nombre' : item_name, 
                                                      'Item_URL_XML' : item_urlXml},
                                                      ignore_index=True)


In [131]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Acuerdos internacionales administrativos],[Corrección de errores del Acuerdo Internacion...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15061],[I. Disposiciones generales]
1,"[MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA...",[Aviación civil],"[Resolución de 6 de noviembre de 2020, de la D...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15062],[I. Disposiciones generales]
2,[MINISTERIO DE TRABAJO Y ECONOMÍA SOCIAL],[Formación profesional para el empleo],"[Orden TES/1109/2020, de 25 de noviembre, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15063],[I. Disposiciones generales]
3,"[MINISTERIO DE INDUSTRIA, COMERCIO Y TURISMO]",[Subvenciones],"[Real Decreto 1040/2020, de 24 de noviembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15064],[I. Disposiciones generales]
4,"[MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN]",[Pesca marítima],"[Real Decreto 968/2020, de 10 de noviembre, po...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15065],[I. Disposiciones generales]
5,[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Moneda metálica. Acuñación],"[Orden ETD/1110/2020, de 25 de noviembre, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15066],[I. Disposiciones generales]
6,[MINISTERIO DE SANIDAD],[Subvenciones],"[Real Decreto 1021/2020, de 17 de noviembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15067],[I. Disposiciones generales]
7,[TRIBUNAL SUPREMO],[Sentencias],"[Sentencia de 29 de septiembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15068],[I. Disposiciones generales]
8,[TRIBUNAL SUPREMO],[Sentencias],"[Sentencia de 15 de octubre de 2020, de la Sal...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15069],[I. Disposiciones generales]


# Descarga ficheros XML asociados

In [132]:
for item_URL in tabla_resumen['Item_URL_XML']:
    #print(URL)
    r = requests.get(item_URL)
    f = './BOEs/' + item_URL[-16:] + '.xml'
    save_html(r.content, f)

In [133]:

import glob
BOEs = glob.glob('./BOEs/BOE*.xml')
print (BOEs)

['./BOEs/BOE-A-2020-14465.xml', './BOEs/BOE-A-2020-14459.xml', './BOEs/BOE-A-2020-14458.xml', './BOEs/BOE-A-2020-14464.xml', './BOEs/BOE-A-2020-14466.xml', './BOEs/BOE-A-2020-14880.xml', './BOEs/BOE-A-2020-14467.xml', './BOEs/BOE-A-2020-14463.xml', './BOEs/BOE-A-2020-14462.xml', './BOEs/BOE-A-2020-14460.xml', './BOEs/BOE-A-2020-14879.xml', './BOEs/BOE-A-2020-14878.xml', './BOEs/BOE-A-2020-14461.xml', './BOEs/BOE-A-2020-14957.xml', './BOEs/BOE-A-2020-14956.xml', './BOEs/BOE-A-2020-15069.xml', './BOEs/BOE-A-2020-15068.xml', './BOEs/BOE-A-2020-14962.xml', './BOEs/BOE-A-2020-15063.xml', './BOEs/BOE-A-2020-15062.xml', './BOEs/BOE-A-2020-14963.xml', './BOEs/BOE-A-2020-14961.xml', './BOEs/BOE-A-2020-15061.xml', './BOEs/BOE-A-2020-14960.xml', './BOEs/BOE-A-2020-14964.xml', './BOEs/BOE-A-2020-14958.xml', './BOEs/BOE-A-2020-15065.xml', './BOEs/BOE-A-2020-15064.xml', './BOEs/BOE-A-2020-14959.xml', './BOEs/BOE-A-2020-14965.xml', './BOEs/BOE-A-2020-15066.xml', './BOEs/BOE-A-2020-15067.xml', './BOEs

In [134]:
#import os
#path = './BOEs'
#BOEs = [f for f in os.listdir(path) if f.startswith('BOE')]
#BOEs

In [135]:
tabla_materias = pd.DataFrame()

for BOE in BOEs:
    #print (BOE)
    BOE_XML = etree.parse(BOE)
    raiz=BOE_XML.getroot()

    materias = [materia.text for materia in raiz.findall('analisis/materias/materia')]
    alertas = [alerta.text for alerta in raiz.findall('analisis/alertas/alerta')]
    #Item_Id = '[' + BOE[7:23] + ']'
    #Item_Id = raiz.find('metadatos/identificador')
    Item_Id = raiz.xpath(".//identificador/text()")
 
    #print(Item_Id, materias, alertas)
    #tabla_materias.loc[['Item_id'], ['Materias']] = materias
    tabla_materias = tabla_materias.append({'Item_id': Item_Id,
                                            'Materias' : materias,
                                            'Alertas' : alertas},
                                            ignore_index=True)

    #tabla_materias.Item_id.apply(list)
    #fila_tabla = tabla_resumen.set_index('Item_id').filter(like=BOE[18:23], axis=0).reset_index()
    #print(fila_tabla['Item_id'])

    #tabla_resumen.loc[fila_tabla['Item_id']]['Materias'] = materias


In [136]:
tabla_materias.sort_values('Item_id')

,Alertas,Item_id,Materias
35,"[Relaciones internacionales, Sanidad, Sistema ...",[BOE-A-2020-14457],"[Acuerdos internacionales, Créditos Presupuest..."
2,"[Educación y enseñanza, Relaciones internacion...",[BOE-A-2020-14458],"[Acuerdos internacionales, Barcelona, Centros ..."
1,"[Educación y enseñanza, Relaciones internacion...",[BOE-A-2020-14459],"[Acuerdos internacionales, Barcelona, Centros ..."
9,"[Administración de Justicia, Sistema financiero]",[BOE-A-2020-14460],"[Ayuntamientos, Créditos Presupuestarios, Juzg..."
12,[Sistema financiero],[BOE-A-2020-14461],[Administración y Contabilidad de la Hacienda ...
8,"[Comercio, Educación y enseñanza, Industria, S...",[BOE-A-2020-14462],[Catálogo Nacional de Cualificaciones Profesio...
7,"[Agricultura, Educación y enseñanza, Ganadería...",[BOE-A-2020-14463],"[Agricultura, Catálogo Nacional de Cualificaci..."
3,"[Educación y enseñanza, Medio ambiente, Seguri...",[BOE-A-2020-14464],"[Bomberos, Catálogo Nacional de Cualificacione..."
0,"[Educación y enseñanza, Pesca, Trabajo y emple...",[BOE-A-2020-14465],"[Actividades subacuáticas, Acuicultura, Catálo..."
4,"[Comercio, Sanidad]",[BOE-A-2020-14466],"[Comercialización, Epidemias, Material sanitar..."


In [137]:
print(raiz.tag)

documento


In [138]:
for materias in raiz.xpath('.//materias'):
    materia = seccion.xpath('.//materia/text()')
    print(materia)

[]


In [139]:
materias=raiz.findall("analisis/materias/materia")
for materia in materias:
    print(materia.text)

Acuerdos internacionales
Créditos Presupuestarios
Ministerio de Sanidad
Organización Mundial de la Salud
Trasplantes de órganos


In [140]:
alertas=raiz.findall("analisis/alertas/alerta")
for alerta in alertas:
    print(alerta.text)

Relaciones internacionales
Sanidad
Sistema financiero


In [141]:
#tabla_resumen.loc[tabla_resumen['Item_id'] == '[' + BOE[7:23] + ']']
print(tabla_resumen.loc[tabla_resumen['Item_id'] == '[BOE-A-2020-14457]'])

Empty DataFrame
Columns: [Departamento, Epigrafe, Item_Nombre, Item_URL_XML, Item_id, Seccion]
Index: []


In [142]:
BOE[7:23]

'BOE-A-2020-14457'

In [143]:
tabla_resumen['Item_id']

0    [BOE-A-2020-15061]
1    [BOE-A-2020-15062]
2    [BOE-A-2020-15063]
3    [BOE-A-2020-15064]
4    [BOE-A-2020-15065]
5    [BOE-A-2020-15066]
6    [BOE-A-2020-15067]
7    [BOE-A-2020-15068]
8    [BOE-A-2020-15069]
Name: Item_id, dtype: object

In [144]:
tabla_materias['Item_id']

0     [BOE-A-2020-14465]
1     [BOE-A-2020-14459]
2     [BOE-A-2020-14458]
3     [BOE-A-2020-14464]
4     [BOE-A-2020-14466]
5     [BOE-A-2020-14880]
6     [BOE-A-2020-14467]
7     [BOE-A-2020-14463]
8     [BOE-A-2020-14462]
9     [BOE-A-2020-14460]
10    [BOE-A-2020-14879]
11    [BOE-A-2020-14878]
12    [BOE-A-2020-14461]
13    [BOE-A-2020-14957]
14    [BOE-A-2020-14956]
15    [BOE-A-2020-15069]
16    [BOE-A-2020-15068]
17    [BOE-A-2020-14962]
18    [BOE-A-2020-15063]
19    [BOE-A-2020-15062]
20    [BOE-A-2020-14963]
21    [BOE-A-2020-14961]
22    [BOE-A-2020-15061]
23    [BOE-A-2020-14960]
24    [BOE-A-2020-14964]
25    [BOE-A-2020-14958]
26    [BOE-A-2020-15065]
27    [BOE-A-2020-15064]
28    [BOE-A-2020-14959]
29    [BOE-A-2020-14965]
30    [BOE-A-2020-15066]
31    [BOE-A-2020-15067]
32    [BOE-A-2020-14875]
33    [BOE-A-2020-14876]
34    [BOE-A-2020-14877]
35    [BOE-A-2020-14457]
Name: Item_id, dtype: object

In [145]:
tabla_resumen.dtypes

Departamento    object
Epigrafe        object
Item_Nombre     object
Item_URL_XML    object
Item_id         object
Seccion         object
dtype: object

In [146]:
tabla_resumen.set_index('Item_id').filter(like='14875', axis=0)

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Seccion
Item_id,,,,,


In [147]:
#tabla_resumen = tabla_resumen.astype(str)
#tabla_resumen['Joined'] = tabla_resumen.Item_id.apply(', '.join)
#tabla_resumen['Item_id']=tabla_resumen['Item_id']

In [148]:
tabla_resumen.dtypes

Departamento    object
Epigrafe        object
Item_Nombre     object
Item_URL_XML    object
Item_id         object
Seccion         object
dtype: object

In [149]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Acuerdos internacionales administrativos],[Corrección de errores del Acuerdo Internacion...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15061],[I. Disposiciones generales]
1,"[MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA...",[Aviación civil],"[Resolución de 6 de noviembre de 2020, de la D...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15062],[I. Disposiciones generales]
2,[MINISTERIO DE TRABAJO Y ECONOMÍA SOCIAL],[Formación profesional para el empleo],"[Orden TES/1109/2020, de 25 de noviembre, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15063],[I. Disposiciones generales]
3,"[MINISTERIO DE INDUSTRIA, COMERCIO Y TURISMO]",[Subvenciones],"[Real Decreto 1040/2020, de 24 de noviembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15064],[I. Disposiciones generales]
4,"[MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN]",[Pesca marítima],"[Real Decreto 968/2020, de 10 de noviembre, po...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15065],[I. Disposiciones generales]
5,[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Moneda metálica. Acuñación],"[Orden ETD/1110/2020, de 25 de noviembre, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15066],[I. Disposiciones generales]
6,[MINISTERIO DE SANIDAD],[Subvenciones],"[Real Decreto 1021/2020, de 17 de noviembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15067],[I. Disposiciones generales]
7,[TRIBUNAL SUPREMO],[Sentencias],"[Sentencia de 29 de septiembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15068],[I. Disposiciones generales]
8,[TRIBUNAL SUPREMO],[Sentencias],"[Sentencia de 15 de octubre de 2020, de la Sal...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-15069],[I. Disposiciones generales]


In [150]:
#for row in tabla_resumen:
#    tabla_resumen.loc[row['Item_id']]['Materias'] = tabla_materias.loc[row['Item_id']]['Materias']

In [151]:
tabla_resumen['Item_id'] = tabla_resumen['Item_id'].astype(str)
tabla_materias['Item_id'] = tabla_materias['Item_id'].astype(str)
tabla_resultados = tabla_resumen.set_index('Item_id').join(tabla_materias.set_index('Item_id'))


In [152]:
tabla_resultados

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Seccion,Alertas,Materias
Item_id,,,,,,,
['BOE-A-2020-15061'],"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Acuerdos internacionales administrativos],[Corrección de errores del Acuerdo Internacion...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Relaciones internacionales, Sanidad, Sistema ...","[Acuerdos internacionales, Créditos Presupuest..."
['BOE-A-2020-15062'],"[MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA...",[Aviación civil],"[Resolución de 6 de noviembre de 2020, de la D...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Trabajo y empleo, Transportes y tráfico]","[Agencias estatales, Descanso laboral, Epidemi..."
['BOE-A-2020-15063'],[MINISTERIO DE TRABAJO Y ECONOMÍA SOCIAL],[Formación profesional para el empleo],"[Orden TES/1109/2020, de 25 de noviembre, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],[Trabajo y empleo],"[Ayudas, Becas, Capacitación profesional, Dese..."
['BOE-A-2020-15064'],"[MINISTERIO DE INDUSTRIA, COMERCIO Y TURISMO]",[Subvenciones],"[Real Decreto 1040/2020, de 24 de noviembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],[Obras y construcciones],"[Congresos, Edificaciones, León, Ministerio de..."
['BOE-A-2020-15065'],"[MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN]",[Pesca marítima],"[Real Decreto 968/2020, de 10 de noviembre, po...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],[Pesca],"[Cantábrico, Cuota de pesca, Material de pesca..."
['BOE-A-2020-15066'],[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Moneda metálica. Acuñación],"[Orden ETD/1110/2020, de 25 de noviembre, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],[Cultura y ocio],"[Monedas conmemorativas, Transportes aéreos]"
['BOE-A-2020-15067'],[MINISTERIO DE SANIDAD],[Subvenciones],"[Real Decreto 1021/2020, de 17 de noviembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],[Sanidad],"[Asistencia farmacéutica, Asistencia sanitaria..."
['BOE-A-2020-15068'],[TRIBUNAL SUPREMO],[Sentencias],"[Sentencia de 29 de septiembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Derecho Administrativo, Transportes y tráfico]","[Autorizaciones, Transporte de viajeros, Trans..."
['BOE-A-2020-15069'],[TRIBUNAL SUPREMO],[Sentencias],"[Sentencia de 15 de octubre de 2020, de la Sal...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],[],[]


In [153]:
tabla_resultados.to_csv('./BOEs/Resultados.csv', index=False)

## Genera Split de ALertas y crea una fila para cada una

### Los items sin Alerta se desechan

In [154]:
tabla_resultados.Alertas.apply(pd.Series) \
    .merge(tabla_resultados, left_index = True, right_index = True) \
    .drop(["Alertas"], axis = 1) \
    .reset_index() \
    .melt(id_vars = ['Item_id','Item_Nombre','Seccion','Departamento','Epigrafe','Item_URL_XML','Materias',], value_name = "Alerta") \
    .drop("variable", axis = 1) \
    .dropna() \
    .sort_values('Alerta')

,Item_id,Item_Nombre,Seccion,Departamento,Epigrafe,Item_URL_XML,Materias,Alerta
5,['BOE-A-2020-15066'],"[Orden ETD/1110/2020, de 25 de noviembre, por ...",[I. Disposiciones generales],[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Moneda metálica. Acuñación],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Monedas conmemorativas, Transportes aéreos]",Cultura y ocio
7,['BOE-A-2020-15068'],"[Sentencia de 29 de septiembre de 2020, de la ...",[I. Disposiciones generales],[TRIBUNAL SUPREMO],[Sentencias],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Autorizaciones, Transporte de viajeros, Trans...",Derecho Administrativo
3,['BOE-A-2020-15064'],"[Real Decreto 1040/2020, de 24 de noviembre, p...",[I. Disposiciones generales],"[MINISTERIO DE INDUSTRIA, COMERCIO Y TURISMO]",[Subvenciones],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Congresos, Edificaciones, León, Ministerio de...",Obras y construcciones
4,['BOE-A-2020-15065'],"[Real Decreto 968/2020, de 10 de noviembre, po...",[I. Disposiciones generales],"[MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN]",[Pesca marítima],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Cantábrico, Cuota de pesca, Material de pesca...",Pesca
0,['BOE-A-2020-15061'],[Corrección de errores del Acuerdo Internacion...,[I. Disposiciones generales],"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Acuerdos internacionales administrativos],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Acuerdos internacionales, Créditos Presupuest...",Relaciones internacionales
6,['BOE-A-2020-15067'],"[Real Decreto 1021/2020, de 17 de noviembre, p...",[I. Disposiciones generales],[MINISTERIO DE SANIDAD],[Subvenciones],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Asistencia farmacéutica, Asistencia sanitaria...",Sanidad
9,['BOE-A-2020-15061'],[Corrección de errores del Acuerdo Internacion...,[I. Disposiciones generales],"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Acuerdos internacionales administrativos],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Acuerdos internacionales, Créditos Presupuest...",Sanidad
18,['BOE-A-2020-15061'],[Corrección de errores del Acuerdo Internacion...,[I. Disposiciones generales],"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Acuerdos internacionales administrativos],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Acuerdos internacionales, Créditos Presupuest...",Sistema financiero
1,['BOE-A-2020-15062'],"[Resolución de 6 de noviembre de 2020, de la D...",[I. Disposiciones generales],"[MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA...",[Aviación civil],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Agencias estatales, Descanso laboral, Epidemi...",Trabajo y empleo
2,['BOE-A-2020-15063'],"[Orden TES/1109/2020, de 25 de noviembre, por ...",[I. Disposiciones generales],[MINISTERIO DE TRABAJO Y ECONOMÍA SOCIAL],[Formación profesional para el empleo],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Ayudas, Becas, Capacitación profesional, Dese...",Trabajo y empleo


In [155]:
tabla_resultados['Epigrafe'].str.join(',')

Item_id
['BOE-A-2020-15061']    Acuerdos internacionales administrativos
['BOE-A-2020-15062']                              Aviación civil
['BOE-A-2020-15063']        Formación profesional para el empleo
['BOE-A-2020-15064']                                Subvenciones
['BOE-A-2020-15065']                              Pesca marítima
['BOE-A-2020-15066']                  Moneda metálica. Acuñación
['BOE-A-2020-15067']                                Subvenciones
['BOE-A-2020-15068']                                  Sentencias
['BOE-A-2020-15069']                                  Sentencias
Name: Epigrafe, dtype: object

In [156]:
tabla_resultados.reset_index(inplace=True)
tabla_resultados['Item_id'].astype(str).str.join('-')

0    [-'-B-O-E---A---2-0-2-0---1-5-0-6-1-'-]
1    [-'-B-O-E---A---2-0-2-0---1-5-0-6-2-'-]
2    [-'-B-O-E---A---2-0-2-0---1-5-0-6-3-'-]
3    [-'-B-O-E---A---2-0-2-0---1-5-0-6-4-'-]
4    [-'-B-O-E---A---2-0-2-0---1-5-0-6-5-'-]
5    [-'-B-O-E---A---2-0-2-0---1-5-0-6-6-'-]
6    [-'-B-O-E---A---2-0-2-0---1-5-0-6-7-'-]
7    [-'-B-O-E---A---2-0-2-0---1-5-0-6-8-'-]
8    [-'-B-O-E---A---2-0-2-0---1-5-0-6-9-'-]
Name: Item_id, dtype: object